In [5]:
# Chess Representation:
#  white: 1
#  black: 2
#  black pieces: pid 1 - 10
#  white pieces: pid 11 - 20

In [6]:
# pid legend:
#       BLACK
# pid:  1      2      3      4      5      6       7       8      9      10
#       queen, king1, king2, rook1, rook2, knight, bishop, pawn1, pawn2, pawn3
# sid:  0      1      1      2      2      3       4       5      5      5

# WHITE
# pid:  11     12     13     14     15     16      17      18     19     20 
#       queen, king1, king2, rook1, rook2, knight, bishop, pawn1, pawn2, pawn3
#       6      7             8             9       10      11

In [7]:
import pygame
import pygame_gui
import random
import sys
import numpy as np

pygame-ce 2.4.0 (SDL 2.28.5, Python 3.11.6)


In [8]:
initial_board = np.array([
    [ 4, 6, 7, 1, 2, 7, 6, 4 ],
    [ 8, 8, 8, 8, 8, 8, 8, 8 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 18, 18, 18, 18, 18, 18, 18, 18 ],
    [ 14, 16, 17, 11, 12, 17, 16, 14 ],
])

In [9]:
#initial_board

In [10]:
# UTILITY FUNCTIONS

In [11]:
# piece id to sprite id
def pid2sid(id):
    assert id in range(1, 21), "Piece ID must be between 0 and 20."
    if id > 10:
        id -= 10
        i = 6
    else:
        i = 0

    if id == 1:
        j = 0
    elif id == 2 or id == 3:
        j = 1
    elif id == 4 or id == 5:
        j = 2
    elif id == 6:
        j = 3
    elif id == 7:
        j = 4
    elif id == 8 or id == 9 or id == 10:
        j = 5
    else:
        assert False, "Logic error."

    return i + j

In [12]:
def player_pid_range(player):
    if player == 1: # white
        return range(11, 21)
    elif player == 2: # black
        return range(1, 11)
    else:
        assert False, "1 and 2 are the only valid player ids."

In [13]:
def piece_player(pid):
    if pid in player_pid_range(1):
        return 1 
    elif pid in player_pid_range(2):
        return 2
    else:
        return 0

In [14]:
# PIECE AND CHESSGAME CLASSES

In [75]:
class Piece():
    def __init__(self, pos, pid):
        self.pos = pos
        self.pid = pid
        self.player = piece_player(pid)
        # ungenerated
        self._type = None
        # self._moves = None

    def __repr__(self):
        return f"Piece(pos={self.pos}, player={self.player}, type={self.type})"

    # @property
    # def moves(self):
    #     assert self._moves is not None, "Cannot get moves of piece if moves haven't been generated."

    # def generate_moves(self):
    #     pass

    @property
    def type(self):
        if self._type:
            return self._type
        else:
            #pid = self.pid
            assert self.pid in range(1, 21), "pid must be in range(1, 21)."
            self._type = self.pid if self.player == 2 else self.pid - 10
            return self._type


    

In [76]:
def move2d(dx, dy):
    return lambda pos: (pos[0] + dx, pos[1] + dy)

In [78]:
class ChessGame():
    def __init__(self, board=None, current_player=1):
        self.board = initial_board.copy() if board is None else board
        self.current_player = current_player
        self._promote_piece = False
        self._move_funcs = { 
            1: self.queen_moves,
            2: self.king_moves,
            3: self.king_moves,
            4: self.rook_moves,
            5: self.rook_moves,
            6: self.knight_moves,
            7: self.bishop_moves,
            8: self.unmoved_pawn_moves,
            #9: self.pawn_moves,
            10: self.pawn_moves
        }
        self._double_pawn_pos = None
        # self._white_positions_cache = None
        # self._black_positions_cache = None
        self._pieces_cache = None
        self._white_pieces_cache = None
        self._black_pieces_cache = None
        self._piece_pos_dict = None
    
    def generate_pieces(self, positions):
        return [ Piece(pos, self.get(pos)) for pos in positions ]
    
    @property
    def white_pieces(self):
        if self._white_pieces_cache:
            return self._white_pieces_cache

        positions = self._white_positions
        self._white_pieces_cache = self.generate_pieces(positions)
        return self._white_pieces_cache

    @property
    def black_pieces(self):
        if self._black_pieces_cache:
            return self._black_pieces_cache

        positions = self._black_positions
        self._black_pieces_cache = self.generate_pieces(positions)
        return self._black_pieces_cache

    @property
    def pieces(self):
        if self._pieces_cache:
            return self._pieces_cache

        self._pieces_cache = self.white_pieces + self.black_pieces
        return self._pieces_cache

    def get_piece(self, pos):
        if self._piece_pos_dict:
            return self._piece_pos_dict[pos]

        self._piece_pos_dict = { piece.pos: piece for piece in self.pieces }
        return self._piece_pos_dict[pos]

    
    def player_positions(self, player):
        return [ (x, y) for y in range(8) for x in range(8) if self.pos_player((x, y)) == player ]
        
    @property
    def _white_positions(self):
        return self.player_positions(1)

    @property
    def _black_positions(self):
        return self.player_positions(2)

    def get_king(self, player):
        target_set = [2, 3] if player == 2 else [12, 13]
        for j in range(8):
            for i in range(8):
                if self.get((j, i)) in target_set:
                    #print(("white" if self.current_player == 1 else "black") + f" king at {(j, i)}")
                    return (j, i)
        return None
    
    def king_safe(self, piece_pos, move_pos, king_pos=None):
        subgame = self.copy()

        # naive "play"
        subgame.move_piece(piece_pos, move_pos)
        subgame.current_player = subgame.opponent

        if king_pos == None:
            king_pos = subgame.get_king(subgame.opponent)
        assert king_pos, "Game is either over or corrupted."
        
        opp_positions = subgame.player_positions(subgame.current_player)

        #print([ (opp_pos, subgame.attack_fn(subgame.piece_type(subgame.get(opp_pos)))(opp_pos)) for opp_pos in opp_positions ])
        
        return king_pos not in [ dest_pos for opp_pos in opp_positions for dest_pos in subgame.attack_fn(subgame.piece_type(subgame.get(opp_pos)))(opp_pos) ]
                        
    def attack_fn(self, type):
        return self._move_funcs[type] if type in range(1,8) else self.pawn_attacks
    
    # @property
    # def _positions(self):
    #      return self._white_positions + self._black_positions                       

    def copy(self):
        return ChessGame(board=self.board.copy(), current_player=self.current_player)

    @property
    def opponent(self):
        return self.current_player ^ 3

    def get(self, pos):
        x, y = pos
        return self.board[y][x]

    def set(self, pos, piece):
        x, y = pos
        self.board[y][x] = piece

    def empty(self, pos):
        return self.get(pos) == 0

    def move_piece(self, src, dst):
        self.set(dst, self.get(src))
        self.set(src, 0)

    def end_turn(self):
        self.current_player = self.opponent

        if self._double_pawn_pos is not None:
            if self.get(self._double_pawn_pos) in [9, 19]: # if pawn still on board (not captured)
                # since we just swapped opponents, if there is a 9 or 19 here, current_player
                # must be back to what it was when we played the 9 or 19.
                self.set(self._double_pawn_pos, 20 if self.current_player == 1 else 10) 
            self._double_pawn_pos = None


    def play(self, piece_pos, move_pos):
        assert self.player_piece(piece_pos), "Bad logic: trying to play move from non-player piece."

        assert self.king_safe(piece_pos, move_pos), "Illegal move."

        x1, y1 = piece_pos
        x2, y2 = move_pos

        piece_type = self.piece_type(self.get(piece_pos))
        
        # Move piece
        if self.get(move_pos) == 0:

            if piece_type == 2 and abs(x1 - x2) == 2: # castle
                if x2 < x1: # long castle
                    self.move_piece((0, y1), (3, y1))
                    self.set((3, y1), self.get((3, y1)) + 1)
                else: # short castle
                    self.move_piece((7, y1), (5, y1))
                    self.set((5, y1), self.get((5, y1)) + 1)
                self.move_piece(piece_pos, move_pos)
                self.set(move_pos, self.get(move_pos) + 1)            

                self.end_turn()
                return
            
            elif piece_type == 8: # unmoved pawn
                x, y1 = piece_pos
                _, y2 = move_pos
                self.set(piece_pos, 0)
                if abs(y1 - y2) == 2: # two spaces
                    self.set(move_pos, 19 if self.current_player == 1 else 9)
                    self.end_turn()
                    self._double_pawn_pos = move_pos
                    return
                else: # one space
                    self.set(move_pos, 20 if self.current_player == 1 else 10)      

            elif piece_type == 10: # normal pawn
                if self.empty(move_pos) and self.piece_type(self.get((x2, y1))) == 9:
                    self.set((x2, y1), 0)
                self.move_piece(piece_pos, move_pos)
            
            else:
                self.move_piece(piece_pos, move_pos)


        # Capture piece
        else:
            if piece_type == 8: # unmoved pawn
                self.set(piece_pos, 0)
                self.set(move_pos, 20 if self.current_player == 1 else 10)      
            else:
                self.move_piece(piece_pos, move_pos)

        if piece_type == 10 and (self.current_player == 1 and y2 == 0 or self.current_player == 2 and y2 == 7):                   
            self._promote_piece = move_pos
            return
        elif piece_type in [2, 4]: # unmoved king or rook
            self.set(move_pos, self.get(move_pos) + 1)

        self.end_turn()
        return
    
    

    @staticmethod
    def in_bounds(pos):
        try:
            x, y = pos
            if not x in range(0, 8) or not y in range(0, 8):
                return False
        except:
            assert False, "pos is not a position."

        return True

    # assert pos is a pos
    # return pid at pos if pos contains one of current_player's live pieces
    # else return None
    def player_piece(self, pos):
        assert self.in_bounds(pos), "Position must be in bounds."
        
        if self.get(pos) in player_pid_range(self.current_player):
            return self.get(pos) # never 0
        else:
            return False
    
    def pos_player(self, pos):
        return piece_player(self.get(pos))
        
    @staticmethod
    def piece_type(pid):
        assert pid in range(1, 21), "pid must be in range(1, 21)."
        if pid in range(11, 21):
            pid -= 10
        return pid

    def moves_in_bounds(self, movelist):
        return [ move for move in movelist if self.in_bounds(move) ]

    # return all positions that do not contain one of current_player's pieces
    def player_open_moves(self, movelist):
        return [ move for move in movelist if not self.player_piece(move) ]

    # return all positions that do not contain any piece
    def open_moves(self, movelist):
        return [ move for move in movelist if self.get(move) == 0 ]

    def generate_line(self, origin, movement):
        pos = movement(origin)
        
        line = []
        while self.in_bounds(pos) and self.pos_player(pos) == 0:
            line = line + [pos]
            pos = movement(pos)

        if self.in_bounds(pos) and self.pos_player(pos) == self.opponent:
            line = line + [pos]

        return line

    def rook_moves(self, pos):
        return [ move
                for movelist in [ self.generate_line(pos, move2d(dx, dy)) for (dx, dy) in [ (1, 0), (-1, 0), (0, 1), (0, -1) ] ]
                for move in movelist
               ]
        
    def bishop_moves(self, pos):
        return [ move
                for movelist in [ self.generate_line(pos, move2d(dx, dy)) for (dx, dy) in [ (1, 1), (-1, 1), (1, -1), (-1, -1) ] ]
                for move in movelist
               ]

    def adjacent_moves(self, pos):
        x, y = pos
        return self.player_open_moves(self.moves_in_bounds([
            (x+dx, y+dy)
            for dx in [1, 0, -1]
            for dy in [1, 0, -1]
            if (dx, dy) != (0, 0)
        ]))

    def queen_moves(self, pos):
        return self.rook_moves(pos) + self.bishop_moves(pos) + self.adjacent_moves(pos)
        # contains duplicates
        # I could do return list(set(...)) but I think that's unnecessary and maybe a bit slower.
    
    def king_moves(self, pos):
        return self.adjacent_moves(pos) + self.castle_moves()

    def castle_moves(self):
        castles = []
        kpos = (4, 0) if self.current_player == 2 else (4, 7)
        kx, ky = kpos
        unmoved_king = (2 if self.current_player == 2 else 12)
        unmoved_rook = (4 if self.current_player == 2 else 14)
        if [ self.get((i, ky)) for i in range(0, 5) ] == [ unmoved_rook, 0, 0, 0, unmoved_king ]:
            if self.king_safe(kpos, (kx-1, ky)) and self.king_safe(kpos, (kx-2, ky)):
                castles += [ (2, ky) ]
        if [ self.get((i, ky)) for i in range(4, 8) ] == [ unmoved_king, 0, 0, unmoved_rook ]:
            if self.king_safe(kpos, (kx+1, ky)) and self.king_safe(kpos, (kx+2, ky)):
                castles += [ (6, ky ) ]
        return castles

    def knight_moves(self, pos):
        x, y = pos
        return self.player_open_moves(self.moves_in_bounds(
            [ (x + d1, y + d2) for (d1, d2) in 
             [p for x, y in [(1, 2), (2, 1)] for p in [(x, y), (-x, y), (x, -y), (-x, -y)]]            
            ]
        ))

    def pawn_attacks(self, pos):
        x, y = pos
        dy = (-1 if self.current_player == 1 else 1)
        return [
            (x + dx, y + dy) for dx in [-1, 1] if (self.in_bounds((x + dx, y + dy)) and self.pos_player((x + dx, y + dy)) == self.opponent)
        ] + self.en_passant_attacks(pos)

    def en_passant_attacks(self, pos):
        x, y = pos
        dy = (-1 if self.current_player == 1 else 1)
        return [
            (x + dx, y + dy) for dx in [-1, 1] if (self.in_bounds((x + dx, y + dy)) and self.pos_player((x + dx, y)) == self.opponent and self.piece_type(self.get((x + dx, y))) == 9)
        ]
    
    def unmoved_pawn_moves(self, pos):
        x, y = pos
        pawn_moves = self.pawn_moves(pos)
        if pawn_moves:
            return self.open_moves(self.moves_in_bounds(
                [ (x, y + (-2 if self.current_player == 1 else 2)) ]
            )) + pawn_moves
        else:
            return []

    def pawn_moves(self, pos):
        x, y = pos
        return self.open_moves(self.moves_in_bounds(
            [ (x, y + (-1 if self.current_player == 1 else 1)) ]
        )) + self.pawn_attacks(pos)
    
    # return list of potential moves (for current player) from pos
    def potential_moves(self, pos):
        if not self.player_piece(pos):
            return None
        else:
            try:
                move_fn = self._move_funcs[self.piece_type(self.get(pos))]
            except:
                print(f'unimplemented {self.player_piece(pos)}')
                return None

            king_pos = self.get_king(self.current_player) if self.piece_type(self.get(pos)) not in [2, 3] else None
            
            #return move_fn(pos)
            moves = move_fn(pos)
            valid_moves = list(filter(lambda dst: self.king_safe(pos, dst), moves))
            return valid_moves

    def generate_moves(self):
        for piece in self.pieces:
            piece.moves = self.potential_moves(piece.pos)

    def checkmate(self):
        # # check king in check
        # enemy_positions = self.player_positions(self.opponent)
        # enemy_moves = list(map(lambda p: self.potential_moves(p), enemy_positions))
        # print(enemy_moves)
        # king_pos = self.get_king(self.current_player)
        # capturing_move = None
        # if not king_pos in enemy_moves:
        #     return False

        # just checking no legal moves erroneously counts draws as 
        # checkmates...
        
        # check no legal moves
        player_positions = self.player_positions(self.current_player)
        player_moves = list(map(lambda p: self.potential_moves(p), player_positions))
        for moveset in player_moves:
            if moveset != []:
                return False
        return True

    def random_move(self):
        player_positions = self.player_positions(self.current_player)
        player_move_dests = list(map(lambda p: self.potential_moves(p), player_positions))
        player_moves = [ (src, dst) for (src, dstlist) in zip(player_positions, player_move_dests) for dst in dstlist ]
        (src, dst) = random.choice(player_moves)
        #print((src, dst))
        self.play(src, dst)

    def promote(self, piece_type):
        # we set promote in play by early returning without end_turn()-ing
        # so current_player is still player to promote piece and we must end turn
        # at end
        x, y = self._promote_piece
        self.board[y][x] = piece_type + (10 if self.current_player == 1 else 0)
        self._promote_piece = None
        self.end_turn()
        
        

In [79]:
# Actual game/render loop
# -----------------------

# Initialize Pygame
pygame.init()

# Rendering constants
WIDTH, HEIGHT = 400, 400
TILE_SIZE = WIDTH // 8

promotion_box_bounds = (5, 5, 2*5 + 4*TILE_SIZE, 2*5 + TILE_SIZE)
piece_boxes = [ (10 + i * TILE_SIZE, 10, TILE_SIZE, TILE_SIZE) for i in range(4) ]

# Colors
# WHITE = (255, 255, 255)
# BLACK = (0, 0, 0)

# Game data
game = ChessGame()
cursor = None
potential_moves = None

# for debug
#game._promote_piece = (0, 2)

# Load sprite sheet
spritesheet = pygame.image.load('ChessPiecesArray.png')
sprite_size = spritesheet.get_size()[1]//2 # height / 2
sprites = [
    pygame.transform.scale(
        spritesheet.subsurface(pygame.Rect(j*sprite_size, i*sprite_size, sprite_size, sprite_size)), # Surface of (i, j)-th sprite
        (TILE_SIZE, TILE_SIZE) # size to resize to
    ) for i in range(2) for j in range(6)
]

# Initialize the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple Chess")

# cursor col,row position to pixel location
def pos2pix(pos):
    (r, c) = pos
    return (TILE_SIZE*r, TILE_SIZE*c)

# Main game loop
running = True
while running:    
    # CONTROL
    # ------------------------------------------------------------    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                continue
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:  # Left mouse button click
            x, y = event.pos
            if game._promote_piece:
                for i in range(4):
                    if pygame.Rect(piece_boxes[i]).collidepoint(x, y):
                        if i == 0:
                            game.promote(1)
                        elif i == 1:
                            game.promote(5)
                        elif i == 2:
                            game.promote(7)
                        else:
                            game.promote(6)
                        
            else:
                ccol = x // TILE_SIZE
                crow = y // TILE_SIZE
                last_cursor = cursor
                cursor = (ccol, crow)
                if potential_moves and cursor in potential_moves:
                    game.play(last_cursor, cursor)
                    potential_moves = None
                else:
                    potential_moves = game.potential_moves(cursor)
                    #print(potential_moves)
    
                if game.checkmate():
                    running = False
                    print('black' if game.current_player == 1 else 'white' + ' wins')
                    print(f'final state:\n{game.board}')
                    continue


        


    # UPDATE
    # ------------------------------------------------------------

    # if game.current_player == 2:
    #     if game.checkmate():
    #         running = False
    #         print('black' if game.current_player == 1 else 'white' + ' wins')
    #         print(f'final state:\n{game.board}')
    #         continue

    #     game.random_move()

    # RENDER
    # ------------------------------------------------------------
    for row in range(8):
        for col in range(8):
            # tile colour
            color = (0xd1, 0xe3, 0xff) if (row + col) % 2 == 0 else (0x57, 0x97, 0xff)
            # cursor tile colour
            if (col, row) == cursor and game.player_piece(cursor):
                color = (0x22, 0x88, 0x00)
            # potential_move tile colour
            elif potential_moves and (col, row) in potential_moves:            
                R, G = (0xdd, 0xcc)
                color = (R, G, 0x88) if (row + col) % 2 == 0 else (R, G, 0x44)
            # draw tile
            pygame.draw.rect(screen, color, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE))
            # draw sprites
            if game.board[row][col] in range(1, 21):
                screen.blit(sprites[pid2sid(game.board[row][col])], pos2pix((col, row)))

    if game._promote_piece:
        pygame.draw.rect(screen, (0xff, 0xff, 0xff), promotion_box_bounds)
        screen.blit(sprites[0], piece_boxes[0])
        screen.blit(sprites[2], piece_boxes[1])
        screen.blit(sprites[4], piece_boxes[2])
        screen.blit(sprites[3], piece_boxes[3])
            
        
    
    pygame.display.flip()


pygame.quit()
#sys.exit()        


In [85]:
c = ChessGame()
#c.white_pieces
#c.black_pieces
#c.get_piece((5, 1))
c.generate_moves()
#c.pieces
for p in c.pieces:
    print(f"{p=}, {p.moves}")

p=Piece(pos=(0, 6), player=1, type=8), [(0, 4), (0, 5)]
p=Piece(pos=(1, 6), player=1, type=8), [(1, 4), (1, 5)]
p=Piece(pos=(2, 6), player=1, type=8), [(2, 4), (2, 5)]
p=Piece(pos=(3, 6), player=1, type=8), [(3, 4), (3, 5)]
p=Piece(pos=(4, 6), player=1, type=8), [(4, 4), (4, 5)]
p=Piece(pos=(5, 6), player=1, type=8), [(5, 4), (5, 5)]
p=Piece(pos=(6, 6), player=1, type=8), [(6, 4), (6, 5)]
p=Piece(pos=(7, 6), player=1, type=8), [(7, 4), (7, 5)]
p=Piece(pos=(0, 7), player=1, type=4), []
p=Piece(pos=(1, 7), player=1, type=6), [(2, 5), (0, 5)]
p=Piece(pos=(2, 7), player=1, type=7), []
p=Piece(pos=(3, 7), player=1, type=1), []
p=Piece(pos=(4, 7), player=1, type=2), []
p=Piece(pos=(5, 7), player=1, type=7), []
p=Piece(pos=(6, 7), player=1, type=6), [(7, 5), (5, 5)]
p=Piece(pos=(7, 7), player=1, type=4), []
p=Piece(pos=(0, 0), player=2, type=4), None
p=Piece(pos=(1, 0), player=2, type=6), None
p=Piece(pos=(2, 0), player=2, type=7), None
p=Piece(pos=(3, 0), player=2, type=1), None
p=Piece(pos=

In [10]:
pygame.Rect((1, 2, 3, 4))

Rect(1, 2, 3, 4)

In [16]:
# TODO:
# 
# fix ChessGame.potential_moves to allow for generation of 
#
# Piece
#   add list of moves to Piece
#
#
#
# The way we're currently checking for checkmate sucks. Fix it.
# check for draw? how do we handle draws?
#   break checkmate into in_check and king.moves == []
#
# need to check for threefold repetition draw
# and fifty-move rule

In [12]:
# TESTS

In [13]:
# in_bounds tests
assert (ChessGame.in_bounds(p) for p in zip(range(8), range(8)))
assert not ChessGame.in_bounds((4, 8))
assert not ChessGame.in_bounds((8, 4))
assert not ChessGame.in_bounds((20, 20))
assert not ChessGame.in_bounds((3, -1))
assert not ChessGame.in_bounds((-1, 3))
assert not ChessGame.in_bounds((-20, -20))

In [14]:
c = ChessGame()
print(c._black_positions)
print(list(map(lambda pos:  c.attack_fn(c.piece_type(c.get(pos)))(pos), c._black_positions)))

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[[(1, 0), (0, 1)], [(2, 2), (0, 2), (3, 1)], [(3, 1), (1, 1)], [(4, 0), (2, 0), (3, 1), (4, 1), (2, 1), (4, 1), (4, 0), (3, 1), (2, 1), (2, 0)], [(5, 1), (5, 0), (4, 1), (3, 1), (3, 0)], [(6, 1), (4, 1)], [(7, 2), (5, 2), (4, 1)], [(6, 0), (7, 1)], [(1, 0)], [(0, 0), (2, 0)], [(1, 0), (3, 0)], [(2, 0), (4, 0)], [(3, 0), (5, 0)], [(4, 0), (6, 0)], [(5, 0), (7, 0)], [(6, 0)]]


In [57]:
d = { (a, a+1): a * (a+1) for a in range(10) }
d

{(0, 1): 0,
 (1, 2): 2,
 (2, 3): 6,
 (3, 4): 12,
 (4, 5): 20,
 (5, 6): 30,
 (6, 7): 42,
 (7, 8): 56,
 (8, 9): 72,
 (9, 10): 90}

In [59]:
{ (a, a+1): a * (a+1) for a in [1, 2, 3, 4] + [5, 6, 7, 8] }

{(1, 2): 2,
 (2, 3): 6,
 (3, 4): 12,
 (4, 5): 20,
 (5, 6): 30,
 (6, 7): 42,
 (7, 8): 56,
 (8, 9): 72}

In [74]:
class A:
    def __init__(self):
        pass

a = A()
a.hi = "hi"
a.hi

'hi'